## Topic Modeling and Insights Data Visualizations for ASSW abstracts.

--- 

This notebook uses topic modeling to analyze ASSW's abstracts. The notebook can be used to see how well the assigned sessions match the topics discovered from the abstracts themselves, so the organizers can adjust the sessions to improve how abstracts are assigned.

Note that topic modeling algorithms work best with a large, diverse corpus. The topics can be explored using LDAVis, which displays the topics in an X-Y plot (intertopic distance map). Topic are represented by circles whose areas are proportional to the relative prevalences of the topics in the corpus. In the display the user can enter a topic number (note 1-base numbering vs. 0-base numbering in Topic List below cells 5 and 7); the terms are displayed on the right, ranked by significance (weight). A topic can be selected on the fly by hovering over its circle; clicking selects that topic. A user can also click on a term in the RH panel to show the topics in which that term occurs. 

The slider at the top of the RH panel allows the user to vary the “saliency”, i.e. uniqueness of the terms to that topic. A value of 0.6 is optimal, according to the authors of the algorithm. Blue bars represent overall term frequency while red bars show term frequency within the topic, which will be different when the saliency Is selected be less than one.

Please see the annotated image of the LDAVis display

![](https://raw.githubusercontent.com/USCDataScience/pdi-topics/master/notebooks/pdi/img/pylda.png)


To provide a quick visual representation of the topics the notebook uses word2vec to infer additional terms and then creates wordclouds out of these words.



**Database Schema:**

Example:

```json
doc = {
        "abstract":["Clouds play a key role in the energy balance of the atmosphere due to their radiative effects, and have a critical  influence on the ice sheet's radiation budget. Changes in the glacier system on the Antarctic Peninsula (AP)  have been observed: disintegration of ice shelves, acceleration and thinning of glaciers, variations in the limits  between glacier faces and retreat of glacier fronts... "],
        "entities":["Marta Caballero1 (marta.caballero@fau.de)",
          " Matthias Braun1",
          " Thomas Mölg1 ",
          "1Friedrich Alexander-Universität Erlangen-Nürnberg",
          " Institut für Geographie",
          " Erlangen",
          " Germany "],
        "title":["Evaluation of Satellite Derived Cloud Top Properties on the Antarctic Peninsula "],
        "id":"Tue_30_AC-1_377 ",
        "_version_":1621681835183439872}
```



In [1]:
# Cell 1: Import requirements

import warnings
warnings.filterwarnings('ignore')

import nltk
import spacy
import urllib
import json
import string
import itertools as it
import numpy as np
import pandas as pd
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.models.word2vec import LineSentence
from os import path


# wordcloud dependencies
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')

# Heavy imports
# from gensim.models import KeyedVectors
# pretrained_model = KeyedVectors.load_word2vec_format('models/glove.6B.100d.vec', binary=False)
nlp = spacy.load('en', disable=['tagger', 'ner'])
# nlp = spacy.load('en')



In [2]:
# Cell 3: Querying Solr

# terms = ['ice', 'climate'] to include only abstracts with specified terms
terms = ['*']
entities = ['*']

# Return "page_size" documents with each Solr query until complete
page_size = 1000
cursorMark = '*'

solr_documents = []
solr_root = 'http://integration.pdi-solr.labs.nsidc.org/solr/assw/select?indent=on&'
more_results = True


if terms[0] != '*':
    terms_wirldcard = ['*' + t + '*' for t in terms]
else:
    terms_wirldcard = ['*']

    
if entities[0] != '*':
    entities_wirldcard = ['*' + e + '*' for e in entities]
else:
    entities_wirldcard = ['*']

terms_query = '%20OR%20abstract:'.join(terms_wirldcard)
entities_query = '%20OR%20entities:'.join(entities_wirldcard)

query_string = 'q=(abstract:{}%20AND%20abstract:/.{{2}}.*/%20AND%20NOT%20title:/.{{300}}.*/)' + \
                '%20AND%20(entities:{})&wt=json&rows={}&cursorMark={}&sort=id+asc'
while (more_results):    
    solr_query = query_string.format(terms_query,
                                     entities_query,
                                     page_size,
                                     cursorMark)
    solr_url = solr_root + solr_query
    print('Querying: \n' + solr_url)
    req = urllib.request.Request(solr_url)
    # parsing response
    r = urllib.request.urlopen(req).read()
    json_response = json.loads(r.decode('utf-8'))
    solr_documents.extend(json_response['response']['docs'])
    nextCursorMark = json_response['nextCursorMark']
    if (nextCursorMark == cursorMark):
        more_results = False
        break
    else: 
        cursorMark = nextCursorMark

total_found = json_response['response']['numFound']
print("Processing {0} documents out of {1} total. \n".format(len(solr_documents), total_found))

Querying: 
http://integration.pdi-solr.labs.nsidc.org/solr/assw/select?indent=on&q=(abstract:*%20AND%20abstract:/.{2}.*/%20AND%20NOT%20title:/.{300}.*/)%20AND%20(entities:*)&wt=json&rows=1000&cursorMark=*&sort=id+asc
Querying: 
http://integration.pdi-solr.labs.nsidc.org/solr/assw/select?indent=on&q=(abstract:*%20AND%20abstract:/.{2}.*/%20AND%20NOT%20title:/.{300}.*/)%20AND%20(entities:*)&wt=json&rows=1000&cursorMark=AoEyRnJpXzE5M19PQy0zXzE4MTgg&sort=id+asc
Querying: 
http://integration.pdi-solr.labs.nsidc.org/solr/assw/select?indent=on&q=(abstract:*%20AND%20abstract:/.{2}.*/%20AND%20NOT%20title:/.{300}.*/)%20AND%20(entities:*)&wt=json&rows=1000&cursorMark=AoExVHVlXzQyX0FDLTFfMTAzOSA=&sort=id+asc
Querying: 
http://integration.pdi-solr.labs.nsidc.org/solr/assw/select?indent=on&q=(abstract:*%20AND%20abstract:/.{2}.*/%20AND%20NOT%20title:/.{300}.*/)%20AND%20(entities:*)&wt=json&rows=1000&cursorMark=AoEvV2VkXzlfQUMtMl83ODIg&sort=id+asc
Processing 2427 documents out of 2427 total. 



In [3]:
years = {str(year) for year in range(2000,2020)}

nlp.Defaults.stop_words |= years
nlp.Defaults.stop_words |= ENGLISH_STOP_WORDS

nlp.Defaults.stop_words |= {'data', 'area', 'interesting', 'water', 'region', 'using', 'different', 'science'
                 'change', 'result', 'research', 'technique', 'datum', 'model', 'use',
                 'observation', 'measurement', 'sample', 'study', 'analysis',}

In [7]:
%%time
# Cell 4: Cleaning our documents  
%load_ext line_profiler


ALL_STOP_WORDS = ENGLISH_STOP_WORDS.union(nlp.Defaults.stop_words)

def flatten(top_list):
    for inner in top_list:
        if isinstance(inner, (list,tuple)):
            for j in flatten(inner):
                yield j
        else:
            yield inner

# Function to clean up the documents, lematizes the words to their regular form and removes the stop words.
def clean_document(doc):
    processed_sentences = []
    raw_sentences = []
    for num, sentence in enumerate(doc.sents):
        raw_sentences.append([item for item in str(sentence).split(' ')])
        tokens = [token.lemma_.encode('ascii',errors='ignore').decode().strip().lower() for token in sentence if token.lemma_ not in string.punctuation]
        cleaned_sentence = [token for token in tokens if token not in ALL_STOP_WORDS]
#         cleaned_sentence = [token for token in cleaned_sentence if len(token) > 1 ]
        cleaned_sentence = [token for token in cleaned_sentence if token != '-PRON-']
        processed_sentences.append(cleaned_sentence)
    return (raw_sentences, processed_sentences)

# bigram_corpus will contain our corpus after cleaning it.

unigram_documents = []

texts = [doc['abstract'][0] for doc in solr_documents]
document_list = []

for doc in nlp.pipe(texts, batch_size=200, n_threads=4):
    assert doc.is_parsed
    raw_sentences, cleaned_sentences = clean_document(doc)
    unigram_documents.append(cleaned_sentences)
    document_list


The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
CPU times: user 5min 7s, sys: 4min 55s, total: 10min 3s
Wall time: 1min 31s


In [5]:
# all the corpus in sentences for training
unigram_sentences = []
bigram_corpus = []
bigram_docs = []

for doc in unigram_documents:
    for sentence in doc:
        unigram_sentences.append(sentence)

bigram_model = Phrases(unigram_sentences, min_count=2)
bigram_phraser = Phraser(bigram_model)

for unigram_doc in unigram_documents:
    bigram_sentences = []
    for unigram_sentence in unigram_doc:
        bigram_sentence = ' '.join(bigram_phraser[unigram_sentence])
        bigram_sentences.append(bigram_sentence)
    bigram_docs.append(bigram_sentences)


for bigram_sentdoc in bigram_docs:
    bigram_tokens = []
    bigram_doc = []
    for sentence in bigram_sentdoc:
        bigram_tokens.append(sentence.split())
    bigram_corpus.append(list(flatten(bigram_tokens)))

print(bigram_corpus[2])

df = pd.DataFrame.from_dict(bigram_corpus)

['decade', 'antarctic', 'ice_sheet', 'accumulation', 'pattern', 'exhibit_large', 'regional_difference', 'total', 'ice_sheet', 'mass', 'antarctica', 'decline', 'east_antarctica', 'experience', 'increase', 'ice', 'mass', 'increase', 'cause', 'handful', 'intense_precipitation', 'event', 'investigate', 'episodic', 'accumulation', 'event', 'explore', 'linkage', 'oceanic', 'evaporation', 'subtropical', 'antarctic', 'ice_sheet', 'accumulation', 'eulerian', 'lagrangian', 'demonstrate', 'moisture_transport', 'east_antarctica', 'facilitate', 'cyclone', 'scale', 'moisture_transport', 'occur', 'coherent', 'air', 'stream', 'manifest', 'initially', 'intense', 'low_level', 'jet', 'embed', 'warm', 'sector', 'cyclone(s', 'follow', 'transition', 'slantwise', 'isentropic', 'ascent', 'reach', 'continent', 'vapor', 'load', 'air', 'stream', 'drive', 'low_level', 'convergence', 'warm', 'sector', 'advance', 'cold', 'slantwise', 'ascent', 'phase', 'moisture_transport', 'characterize', 'moisture', 'removal', 'p

 **Building the LDA model using bigrams**

In [8]:
# Cell 5: LDA Topic Modeling

# num pases should be adjusted, 3 is just a guesstimate of when convergence will be achieved.
num_passes = 3
num_topics = 20
words_per_topic = 7

dictionary = corpora.Dictionary(bigram_corpus)
lda_corpus = [dictionary.doc2bow(text) for text in bigram_corpus]

lda_model = LdaMulticore(lda_corpus,
                         num_topics=num_topics,
                         id2word=dictionary,
                         passes=num_passes,
                         workers=2
                        )
topics = lda_model.print_topics(num_topics=num_topics, num_words=words_per_topic)
print ("Topic List: \n")
for topic in topics:
    t = str((int(topic[0])+ 1))
    print('Topic ' + t + ': ', topic[1:])

import warnings
warnings.filterwarnings('ignore')

import pyLDAvis.gensim
print ("\nPyLDAVis: \n")
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(corpus=lda_corpus,
                        topic_model=lda_model,
                        dictionary=dictionary,
                        sort_topics=False)

# we recomend to adjust lambda to 0.6 as is recommended by the paper authors.

Topic List: 

Topic 1:  ('0.009*"arctic" + 0.006*"ice" + 0.005*"surface" + 0.005*"process" + 0.005*"include" + 0.004*"sea" + 0.004*"sea_ice"',)
Topic 2:  ('0.009*"arctic" + 0.006*"glacier" + 0.006*"sea_ice" + 0.005*"antarctic" + 0.005*"ocean" + 0.005*"present" + 0.005*"antarctica"',)
Topic 3:  ('0.010*"glacier" + 0.006*"ice_core" + 0.004*"project" + 0.004*"high" + 0.004*"change" + 0.004*"ice" + 0.003*"record"',)
Topic 4:  ('0.013*"ice" + 0.008*"change" + 0.006*"ice_sheet" + 0.006*"snow" + 0.005*"glacier" + 0.005*"climate" + 0.004*"antarctic"',)
Topic 5:  ('0.016*"sea_ice" + 0.009*"ocean" + 0.009*"change" + 0.007*"increase" + 0.007*"arctic" + 0.006*"variability" + 0.005*"surface"',)
Topic 6:  ('0.012*"change" + 0.009*"glacier" + 0.006*"increase" + 0.005*"high" + 0.005*"temperature" + 0.005*"ice" + 0.003*"antarctic"',)
Topic 7:  ('0.009*"antarctic" + 0.006*"change" + 0.005*"antarctica" + 0.005*"arctic" + 0.004*"polar" + 0.004*"ice" + 0.003*"sea_ice"',)
Topic 8:  ('0.011*"arctic" + 0.005*

/home/beto/.pyenv/versions/miniconda3-latest/envs/pdi-topics/lib/python3.5/site-packages/funcy-1.7.2-py3.5.egg/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
/home/beto/.pyenv/versions/miniconda3-latest/envs/pdi-topics/lib/python3.5/site-packages/funcy-1.7.2-py3.5.egg/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
/home/beto/.pyenv/versions/miniconda3-latest/envs/pdi-topics/lib/python3.5/site-packages/funcy-1.7.2-py3.5.egg/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
/home/beto/.pyenv/versions/miniconda3-latest/envs/pdi-topics/lib/python3.5/site-packages/funcy-1.7.2-py3.5.egg/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
/home/beto/.pyenv/versions/miniconda3-latest/envs/pdi-topics/lib/python3.5/site-packages/funcy-1.7.2-py3.5.e

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
0      6.236517        1       1 -0.019490  0.003660
1      5.780824        1       2 -0.012725  0.000307
2      4.530452        1       3  0.025473 -0.066055
3      7.124140        1       4 -0.013533 -0.068255
4      6.110833        1       5 -0.055946  0.068862
5      4.830516        1       6 -0.019015 -0.019783
6      4.861359        1       7  0.031947  0.044579
7      3.213179        1       8  0.011955  0.006626
8      5.990768        1       9 -0.028277  0.020234
9      2.878458        1      10 -0.012488 -0.031876
10     4.330649        1      11  0.038978  0.041440
11     1.808625        1      12  0.104296  0.009358
12     8.208921        1      13 -0.037592 -0.041325
13     2.927637        1      14  0.002387 -0.043851
14     3.477796        1      15  0.001441 -0.031379
15     7.744125        1      16 -0.016254  0.051084
16     7.877700        1      17 -0.084329  0.012761
17     3.682280        1      18  0.059820  0.006747
18     5.337132        1      19  0.010714 -0.012060
19     3.048090        1      20  0.012639  0.048925, topic_info=      Category         Freq            Term        Total  loglift  logprob
term                                                                      
8338   Default  1901.000000         sea_ice  1901.000000  30.0000  30.0000
10854  Default  1624.000000          arctic  1624.000000  29.0000  29.0000
21960  Default   765.000000         glacier   765.000000  28.0000  28.0000
26465  Default  1851.000000             ice  1851.000000  27.0000  27.0000
14626  Default  1269.000000       antarctic  1269.000000  26.0000  26.0000
19822  Default   272.000000            lake   272.000000  25.0000  25.0000
25506  Default   665.000000            snow   665.000000  24.0000  24.0000
2842   Default  1495.000000          change  1495.000000  23.0000  23.0000
19909  Default   503.000000         species   503.000000  22.0000  22.0000
1830   Default   772.000000           ocean   772.000000  21.0000  21.0000
26035  Default   287.000000        ice_core   287.000000  20.0000  20.0000
10604  Default   125.000000           fjord   125.000000  19.0000  19.0000
24487  Default   240.000000       ice_shelf   240.000000  18.0000  18.0000
22402  Default   438.000000            melt   438.000000  17.0000  17.0000
12187  Default   649.000000     temperature   649.000000  16.0000  16.0000
7196   Default   233.000000           cloud   233.000000  15.0000  15.0000
20263  Default   890.000000            high   890.000000  14.0000  14.0000
19059  Default   803.000000        increase   803.000000  13.0000  13.0000
4094   Default   461.000000             sea   461.000000  12.0000  12.0000
20484  Default   572.000000       ice_sheet   572.000000  11.0000  11.0000
13365  Default   128.000000            wave   128.000000  10.0000  10.0000
18180  Default   858.000000         surface   858.000000   9.0000   9.0000
6476   Default   381.000000     atmospheric   381.000000   8.0000   8.0000
21641  Default   284.000000            soil   284.000000   7.0000   7.0000
23515  Default   342.000000  southern_ocean   342.000000   6.0000   6.0000
14290  Default   261.000000               c   261.000000   5.0000   5.0000
17325  Default   120.000000           drill   120.000000   4.0000   4.0000
16141  Default   422.000000         project   422.000000   3.0000   3.0000
6389   Default   523.000000            warm   523.000000   2.0000   2.0000
487    Default   801.000000      antarctica   801.000000   1.0000   1.0000
...        ...          ...             ...          ...      ...      ...
24034  Topic20    23.605486         compare   406.461975   0.6446  -5.8048
4340   Topic20    24.361986     variability   467.743103   0.5358  -5.7733
2350   Topic20    11.491143          series    95.633942   1.3717  -6.5247
6653   Topic20    29.316536           polar   747.149841   0.2525  -5.5882
16001  Topic20  

In [ ]:
# Cell 6: Classifying an abstract using our GENSIM model

# ESSI abstracts taken from https://meetingorganizer.copernicus.org/EGU2018/EGU2018-9778.pdf

document = """
Submarine slope failure is a ubiquitous process and dominant pathway for sediment and organic carbon ﬂux from 
continental margins to the deep sea. Slope failure occurs over a wide range of temporal and spatial scales, 
from small (10e4-10e5 m3/event), sub-annual failures on heavily sedimented river deltas to margin-altering and 
tsunamigenic (10-100 km3/event) open slope failures occurring on glacial-interglacial timescales. 
Despite their importance to basic (closing the global source-to-sink sediment budget) and applied 
(submarine geohazards) re- search, submarine slope failure frequency and magnitude on most continental margins 
remains poorly constrained. This is primarily due to difﬁculty in 1) directly observing events, and 2) reconstructing 
age and size, particularly in the geologic record. The state of knowledge regarding submarine slope failure 
preconditioning and triggering factors is more qualitative than quantitative; a vague hierarchy of factor importance 
has been established in most settings but slope failures cannot yet be forecasted or hindcasted from 
a priori knowledge of these factors.
"""

vec = dictionary.doc2bow(clean_document(nlp(document)))
predicted_topics = lda_model[vec]
predicted_topics = [(p[0]+1, p[1]) for p in predicted_topics]
print(predicted_topics)

### References and Links



> L. A. Lopez, R. Duerr and S. J. S. Khalsa, "Optimizing apache nutch for domain specific crawling at large scale," 2015 IEEE International Conference on Big Data (Big Data), Santa Clara, CA, 2015, pp. 1967-1971.
doi: 10.1109/BigData.2015.7363976

> Sievert, C & Shirley, K.E.. (2014). LDAvis: A method for visualizing and interpreting topics. Proceedings of the Workshop on Interactive Language Learning, Visualization, and Interfaces. 63-70. 

> Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. [pdf] [bib] 